# Dash

In [2]:
#Load the package
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

## A Very Simple Dash App

In [ ]:
#Create the dash app
app = Dash(__name__)

#Define the layout of the app
app.layout = html.Div([
    html.H1("Welcome to my Dashboard!"), #A level 1 header
    html.P("This is a simple example."), #A paragraph of the text
    html.Div("This is another div.") #Another div element
])

#Run the app
app.run_server(debug = True)

## Steps to Create a Dash Application

In [ ]:
#Step 1: Import the necessary libraries
#Load the dataset
df_pm25 = pd.read_csv("I:/My Drive/113 Fall 2024 PDDS/PDDS W11/TaiwanPM25_synthetic_data-1.csv")

#Melt the data for Plotly Compatibility
df_pm25_long = df_pm25.melt(id_vars = 'Year',
                       var_name = 'Region',
                       value_name = 'PM2.5')
print(df_pm25_long)

#Step 2: Create the Dash Application
app_pm25 = Dash(__name__)

#For deployment
server = app.server

#Step 3: Set up the HTML layout
app_pm25.layout = html.Div([
    html.H2("PM2.5 levels Across Taiwan Regions (2010 - 2022)"), #Header
    dcc.Dropdown(
        id = 'dropdown',
        options = [{'label': region, 'value': region}
                   for region in df_pm25_long['Region'].unique()],
        value = 'Northern_Taiwan', #Default selection
        multi = True #Allow multiple region selection dropdown for selecting regions
    ),
    dcc.Graph(id = "graph") #Graph component for Plotly Chart
])

#Step 4: Add callback function
@app_pm25.callback(Output("graph", "figure"), [Input('dropdown', 'value')])
def update_chart(selected_regions):
    #Ensure selected regions is a list, even if only one region is selected, isinstance() is a function used to check if an object of a specific type or class
    if isinstance(selected_regions, str):
        selected_regions = [selected_regions]
    
    #Filter data based on selected regions
    filtered_df_pm25 = df_pm25_long[df_pm25_long['Region'].isin(selected_regions)]

    #Generate the line plot
    fig_pm25 = px.line(
        filtered_df_pm25,
        x = "Year",
        y = "PM2.5",
        color = 'Region',
        title = "PM2.5 Levels Across Taiwan Regions (2010 - 2022)",
        markers = True #Add markers to each data point for better readability
    )

    #Customize the layout
    fig_pm25.update_layout(
        yaxis_title = "PM2.5 (miug/m3)",
        xaxis_title = "Year",
        legend_title_text = "Region"
    )

    #Set the y-axis to start at a minimum level, not zero, for better detail on small changes
    fig_pm25.update_yaxes(rangemode = "tozero", title = "PM2.5 (miug/m3)")

    return fig_pm25

#Step 5: Run the application
if __name__ == '__main__':
    app_pm25.run(debug = True)

    Year           Region  PM2.5
0   2010  Northern_Taiwan     35
1   2011  Northern_Taiwan     34
2   2012  Northern_Taiwan     36
3   2013  Northern_Taiwan     35
4   2014  Northern_Taiwan     33
..   ...              ...    ...
86  2018  Hualien_Taitung     13
87  2019  Hualien_Taitung     12
88  2020  Hualien_Taitung     11
89  2021  Hualien_Taitung     10
90  2022  Hualien_Taitung      9

[91 rows x 3 columns]
